HW1 solution
======

## Behavioural cloning

TODO:
----
1. generate rollouts
This are sequences of the type expertSequence#1 = {s0 -> a0, s1 -> a1, s2 -> a3} done with the expert policy
<br>

- expert_data/something.pkl
- big number -> 18101
- observations.shape = (big number, 44)
- actions.shape = (big number, 17)
- returns.shape = (20, )
<br>
----
2. implement behavioural cloning
implement a neural net of the sort F(s) = a trained on all the expertSequence. Then create a policy using the neural net

In [1]:
import pickle as pk

In [2]:
expert_rollouts = pk.load(open("expert_data/RoboschoolHumanoid-v1.pkl","rb"))

In [3]:
expert_rollouts

{'observations': array([[ 2.3841858e-08,  1.7628326e-01,  9.8433948e-01, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [-2.1484853e-03,  1.6638857e-01,  9.8606026e-01, ...,
         -7.1955717e-01,  0.0000000e+00,  0.0000000e+00],
        [-8.5775014e-03,  1.4194125e-01,  9.8987508e-01, ...,
         -6.6612923e-01,  0.0000000e+00,  0.0000000e+00],
        ...,
        [-4.0738460e-01, -1.3294163e-01,  9.9112386e-01, ...,
          3.7752178e-02,  1.0000000e+00,  0.0000000e+00],
        [-4.0473124e-01, -2.1884102e-01,  9.7576052e-01, ...,
          1.5287129e-02,  1.0000000e+00,  0.0000000e+00],
        [-4.0661630e-01, -3.0624390e-01,  9.5195311e-01, ...,
          6.2616086e-03,  1.0000000e+00,  0.0000000e+00]], dtype=float32),
 'actions': array([[-0.82425266,  0.85833374,  0.25510112, ..., -0.63416464,
         -0.02664589, -0.3858894 ],
        [-0.55048927,  0.23574406,  0.66313531, ..., -0.77063167,
         -0.04402356,  0.10912636],
        [ 0.3713653

In [4]:
expert_rollouts['observations'].shape

(18091, 44)

In [5]:
expert_rollouts['actions'].shape

(18091, 17)

In [6]:
expert_rollouts['returns'].shape

(20,)

In [7]:
inputs = expert_rollouts['observations']
outputs = expert_rollouts['actions']

In [8]:
assert len(inputs) == len(outputs)

In [9]:
split = int(len(inputs)*0.8)
print(split)

14472


In [10]:
X_train, y_train = inputs[:split], outputs[:split]
X_test, y_test = inputs[split:], outputs[split:]

model
========

In [11]:
import tensorflow as tf
import numpy as np

In [12]:
def tf_reset():
    try:
        sess.close()
    except:
        pass
    tf.reset_default_graph()
    return tf.Session()

In [13]:
sess = tf_reset()
def create_model():
    input_ph = tf.placeholder(dtype=tf.float32, shape=[None, 44]) # [None, 1] is because is 1D array
    output_ph = tf.placeholder(dtype=tf.float32, shape=[None, 17])
    
    W0 = tf.get_variable(name='W0', shape=[44, 30], initializer = tf.contrib.layers.variance_scaling_initializer())
    W1 = tf.get_variable(name='W1', shape=[30, 23], initializer = tf.contrib.layers.variance_scaling_initializer())
    W2 = tf.get_variable(name='W2', shape=[23, 17], initializer = tf.contrib.layers.variance_scaling_initializer())
    
    
    b0 = tf.get_variable(name='b0', shape=[30], initializer = tf.constant_initializer(0))
    b1 = tf.get_variable(name='b1', shape=[23], initializer = tf.constant_initializer(0))
    b2 = tf.get_variable(name='b2', shape=[17], initializer = tf.constant_initializer(0))
    
    weights = [W0, W1, W2]
    biases = [b0, b1, b2]
    activations = [tf.nn.relu, tf.nn.relu, None]
    
    layer = input_ph
    for W, b, activation in zip(weights, biases, activations):
        layer = tf.matmul(layer, W) + b
        if activation is not None:
            layer = activation(layer)
            
    output_pred = layer
    return input_ph, output_ph, output_pred

In [14]:
input_ph, output_ph, output_pred = create_model()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [15]:

mse = tf.reduce_mean(0.5 * tf.square(output_pred - output_ph)) # this is the mean square error
opt = tf.train.AdamOptimizer().minimize(mse) # this is an operation that pereform gradient descent

In [16]:

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver() # save weight as the training goes on

In [17]:
#training
batch_size = 32
training_mse = []
for training_step in range(5000):
    #random batch
    indices = np.random.randint(low = 0, high = len(X_train), size = batch_size)
    input_batch = X_train[indices]
    output_batch = y_train[indices]
    
    # run optimizer and get mse
    _, mse_run = sess.run([opt, mse], feed_dict={input_ph: input_batch, output_ph: output_batch})
    
    training_mse.append(mse_run)
    if training_step % 1000 == 0:
        print('{0:04d} mse : {1:.3f}'.format(training_step, mse_run))
        saver.save(sess, '/tmp/model.ckpt')

0000 mse : 0.458
1000 mse : 0.031
2000 mse : 0.026
3000 mse : 0.028
4000 mse : 0.020


In [18]:
from matplotlib import pyplot as plt
plt.figure(figsize=(15,10))
plt.plot(training_mse)
plt.show()

<Figure size 1500x1000 with 1 Axes>

In [20]:
sess = tf_reset()

input_ph, output_ph, output_pred = create_model()
saver = tf.train.Saver() 
saver.restore(sess, "/tmp/model.ckpt")

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


In [21]:
# validation score was 0.03

In [22]:
# prediction of only one action
output_pred_run = sess.run(output_pred, feed_dict={input_ph: X_test[0].reshape((1,44))})

In [23]:
output_pred_run

array([[ 0.42027014,  0.30810767, -0.6921607 ,  0.66746587, -0.08639976,
        -0.9280569 , -0.12618512,  0.49747017,  0.6034339 ,  1.0716732 ,
         0.9731382 , -0.18995148, -0.22227001, -0.17566441, -0.5313878 ,
        -0.36136973, -0.36491805]], dtype=float32)

In [24]:
y_test[0]

array([ 0.44276008,  0.21976118, -0.57139829,  0.80728258, -0.17347842,
       -0.86538909, -0.21637144,  0.33584227,  0.4801961 ,  1.11101721,
        0.91422874, -0.32985784, -0.28811901, -0.10695498, -0.45460487,
       -0.28110336, -0.49649675])

In [25]:
mse = ((output_pred_run[0] - y_test[0])**2).mean(axis=None)

In [26]:
mse

0.009805779074173392

In [27]:
import gym, roboschool

In [28]:
env=gym.make("RoboschoolHumanoid-v1")

/Users/alex/Desktop/Coding/AI/CS294_DeepReinforcementLearning/homeworks/hw1/venv/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [29]:
env

<TimeLimit<RoboschoolHumanoid<RoboschoolHumanoid-v1>>>

In [30]:
import os;os.listdir('.')

['.DS_Store',
 '.ipynb_checkpoints',
 '.mypy_cache',
 '.run_expert.py.swp',
 '__pycache__',
 'demo.bash',
 'Dockerfile',
 'expert_data',
 'experts',
 'imitation_trainer',
 'load_policy.py',
 'README.md',
 'report.ipynb',
 'report.md',
 'requirements.txt',
 'run_expert.py',
 'template_trainer',
 'tf_util.py',
 'venv']

In [31]:
import run_expert

In [32]:
class Policy():
    def __init__(self):
        self.sess = tf_reset()

        self.input_ph, self.output_ph, self.output_pred = create_model()
        saver = tf.train.Saver() 
        saver.restore(self.sess, "/tmp/model.ckpt")
        
    def act(self, obs):
        return self.sess.run(self.output_pred, feed_dict={self.input_ph: obs.reshape((1,44))})[0]

In [33]:
policy=Policy()

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


In [34]:
run_expert.run_policy(env,policy,20,"running trained model",render=True)

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
Env description: running trained model
mean return 19.445062228252716
std of return 22.03125475034104


{'observations': array([[ 6.0000002e-01, -1.3641670e-01,  9.9065155e-01, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 5.9764951e-01, -1.0928621e-01,  9.9401033e-01, ...,
         -6.5446183e-02,  0.0000000e+00,  0.0000000e+00],
        [ 5.9032744e-01, -8.8491201e-02,  9.9607694e-01, ...,
         -3.0168319e-01,  0.0000000e+00,  0.0000000e+00],
        ...,
        [ 3.0823695e-02,  3.8879395e-01,  9.2132473e-01, ...,
          1.9786246e-04,  0.0000000e+00,  1.0000000e+00],
        [ 1.2576728e-02,  4.3675664e-01,  8.9957970e-01, ...,
         -1.2969035e-01,  0.0000000e+00,  1.0000000e+00],
        [-6.9913077e-03,  4.9315590e-01,  8.6994094e-01, ...,
          1.9295053e-01,  0.0000000e+00,  1.0000000e+00]], dtype=float32),
 'actions': array([[ 0.37206176, -0.29462016,  0.5891975 , ..., -0.6602019 ,
          0.29638252, -0.09097595],
        [-1.1759261 , -0.19891082,  0.36412528, ..., -0.0667505 ,
         -0.2717951 , -0.05463134],
        [-0.4070432